# Assignment 1 #
Karthik Karavatt
2061996
5

In [79]:
import cv2 as cv
# Read the image
img = cv.imread("digits.png")
# Convert the image to grayscale
img = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

## Task 1 Data Preparation 

### 1.1 
Split the image into 5000 images of size 20x20 pixels each. Each image contains a digit.
Store this data in a numpy array

We know the digit is 20x20 pixels. So we can split the image into 5000 images of size 20x20 pixels each.
As the image is 1000x500 pixels, we can split the image into 50 rows and 100 columns.

In [80]:
import numpy as np
# Split the image into 5000 cells of size 20x20 pixels each
data_cells = np.array([np.hsplit(row, 100) for row in np.vsplit(img, 50)])

data = np.array(data_cells)

### 1.2
Split the data into training and test sets. Use the first 1000 images for training and the rest for testing.
Every 5th row is a different digit, there are 5 rows of each digit and the digits go up to 9, we can use one row of each digit for training and the rest for testing. So we take one row from every 5 rows for training and the rest for testing
.

In [81]:
# Select every 5th row for testing
test_data = data[::5]

# Select all but every 5th row for training
train_data = np.delete(data, np.arange(0, data.shape[0], 5), axis=0)

### 1.3
Write the images to a folders. The images should be written to the train and test directories.

In [82]:
import os
import shutil
import cv2 as cv

# Define your directory paths
train_dir = 'train'
test_dir = 'test'

# If the directories already exist, remove them and their contents
for dir_path in [train_dir, test_dir]:
    if os.path.exists(dir_path) and os.path.isdir(dir_path):
        shutil.rmtree(dir_path)

    # Create the directories
    os.makedirs(dir_path)

# Save your train and test data to the respective directories as JPEGs

for i in range(train_data.shape[0]):
    for j in range(train_data.shape[1]):
        digit = i // 4  # The digit is determined by integer division of the row index by 4
        row = i % 4  # The row is determined by the remainder of the row index divided by 4
        cv.imwrite(os.path.join(train_dir, f'train_{digit}_{row}_{j}.jpg'), train_data[i, j])

for i in range(test_data.shape[0]):
    for j in range(test_data.shape[1]):
        cv.imwrite(os.path.join(test_dir, f'test_{i}_{j}.jpg'), test_data[i, j])


## Task 2: Nearest Neighbor method for image classification 

In this task we will use the k - nearest neighbor method to classify the images in the test set.
### 2.1
label the images in the training set. The labels should be the digit in the image.

In [83]:
# Reshape the training data to be 2D and convert to float32
train_data = train_data.reshape(-1, 400)
train_data = np.float32(train_data)

# Reshape the test data to be 2D and convert to float32
test_data = test_data.reshape(-1, 400)
test_data = np.float32(test_data)

# Create the labels for the training data
train_labels = np.repeat(np.arange(10), 400).reshape(-1, 1)
# Create the labels for the test data
test_labels = np.repeat(np.arange(10), 100).reshape(-1, 1)


### 2.2 Implement the k - nearest neighbor method to classify the images in the test set.

In [84]:
# Create the kNN model
knn = cv.ml.KNearest_create()
knn.train(train_data, cv.ml.ROW_SAMPLE, train_labels)
ret, result, neighbours, dist = knn.findNearest(test_data, k=1)
# Calculate the accuracy of classification
matches = np.equal(result, test_labels)
correct = np.count_nonzero(matches)
accuracy = correct * (100.0 / result.size)
print(f'Accuracy is {accuracy}%')

Accuracy is 92.60000000000001%
